In [ ]:
import numpy as np
import os
import glob
import locale
import statistics  as st
from openpyxl import Workbook
from requests import request
from bs4 import BeautifulSoup
import geopandas as gpd
import requests
import os
import pandas as pd
import urllib
import requests
import numpy as np
import matplotlib.pyplot as plt
import datetime
import partridge as ptg
import geopandas as gpd
from shapely.geometry import LineString
import time
import folium
from branca.colormap import LinearColormap
from ipyleaflet import WidgetControl
import ipywidgets as widgets
import jenkspy as jenks
import ipywidgets
from folium.plugins import Search
%matplotlib inline
import warnings
import datetime
import json
from folium import plugins
from folium.plugins import MeasureControl
from django.views.generic import TemplateView
import datetime
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots  import make_subplots
warnings.filterwarnings("ignore")
pd.set_option("max_columns",70)
# os.chdir(r"C:\Users\gregory.sk\Desktop")

In [ ]:
def read_data():
    url = "http://moran.mot.gov.il:110/Channels/HTTPChannel/SmQuery/2.8/json?Key=YB01801871&MonitoringRef=AllActiveTripsFilter&StopVisitDetailLevel=normal"
    r =requests.get(url,allow_redirects=True)
    data = json.loads(r.content)
    df_current = pd.DataFrame(data["Siri"]["ServiceDelivery"])
    df_all_trips_current = pd.DataFrame(df_current["StopMonitoringDelivery"][0])
    data_noraml_current= pd.json_normalize(df_all_trips_current['MonitoredStopVisit'])
    listy =["RecordedAtTime"]
    for i in data_noraml_current.columns[1:]:
        listy.append(i.split(".")[-1])
    data_noraml_current.columns = listy
    data_noraml_current.columns= data_noraml_current.columns.str.replace('Ref','')
    data_noraml_current["RecordedAtTime_onlytime"]=data_noraml_current["RecordedAtTime"].str[11:19]
    data_noraml_current = data_noraml_current[~(data_noraml_current["Longitude"].isna()|data_noraml_current["Latitude"].isna())]
    data_noraml_current["Velocity"]=data_noraml_current["Velocity"].astype(int)
    data_noraml_current =data_noraml_current.sort_values(["DatedVehicleJourney","Order"])
    data_noraml_current['DistanceFromStop']=data_noraml_current['DistanceFromStop'].astype(int)
    data_noraml_current.Line=data_noraml_current.Line.astype("int64")
    return data_noraml_current
    
    

In [ ]:
def update_data_sample():
    df_keep1 = pd.DataFrame()
    df_keep1 = pd.concat([df_keep1,read_data()])
    return df_keep1

In [ ]:
def data_merge():
    df_keep1 = update_data_sample()
    lines = pd.read_excel(r"Z:\פרוייקטים\מאגרי מידע\סטטיסטיקה\קבצים מהרישוי\2021\קיץ 3\lines.xlsx")
    lines = lines.rename(columns={"מקט":"Line_code","כיוון":"Direction","שעת יציאה":"Hour","יום":"Day","חלופה":"Alternative"})
    lines_jerus = lines[lines["מס' אשכול"].isin([91,134])]
    lines_alt_list =lines_jerus["מזהה החלופה"].unique()
    df_keep_jerus=df_keep1[df_keep1.Line.isin(lines_alt_list)]
    df_keep_jerus["test_index"] = np.arange(1,len(df_keep_jerus)+1)
    mask = (df_keep_jerus['DatedVehicleJourney'].shift(1)!=df_keep_jerus['DatedVehicleJourney'])
    df_keep_jerus["Arrivalminutes"] =pd.to_datetime(df_keep_jerus.RecordedAtTime_onlytime).dt.minute.astype("int")
#     df_keep_jerus['diff_time'] = np.where(mask, 0,abs(df_keep_jerus['Arrivalminutes'].shift(1)-df_keep_jerus['Arrivalminutes']))
    df_keep_jerus_merge =pd.merge(df_keep_jerus,lines_jerus[["Line_code","Direction","מס' אשכול","קו","שם יישוב מוצא","מזהה החלופה","Alternative"]],left_on="Line",right_on="מזהה החלופה",how="left")
    df_keep_jerus_merge=df_keep_jerus_merge.drop_duplicates("test_index")
    df_keep_jerus_merge=df_keep_jerus_merge[df_keep_jerus_merge["DatedVehicleJourney"]!="0"]
    return df_keep_jerus_merge

In [ ]:
def colorfull():
    df_keep_jerus_merge = data_merge()
    df_keep_jerus_merge["color"] ="green"
    return df_keep_jerus_merge

In [ ]:
def compare_to_normal():
    df_keep_jerus_merge = colorfull()
    df_normal = pd.read_csv(r"G:\python_scripts2\real_time_data_siri_project\data_compare\lines_data_november.csv")
    df_normal = df_normal[df_normal["ClusterId"].isin([91,134])].copy()
    df_keep_jerus_merge["StopPoint"] =df_keep_jerus_merge["StopPoint"].astype("int64")
    df_keep_jerus_merge["Order"] =df_keep_jerus_merge["Order"].astype("int64")
    df_normal["StopSequence"] = df_normal["StopSequence"]+1
    df_keep_jerus_merge=pd.merge(df_keep_jerus_merge,df_normal[["ClusterId","day_time","RouteShortName","Direction","StopSequence","StopCode","StopName","prop_in_time","average_time_travel_to_next_stop","median_time_travel_to_next_stop","speed_to_next_stop","sd_dif","tikufim_cat","dist_to_next_stop","average_speed"]],left_on=["מס' אשכול","קו","Direction","StopPoint","Order"],right_on=["ClusterId","RouteShortName","Direction","StopCode","StopSequence"],how="left")
    df_keep_jerus_merge = df_keep_jerus_merge.drop_duplicates("test_index")
    df_keep_jerus_merge =df_keep_jerus_merge[~df_keep_jerus_merge["speed_to_next_stop"].isna()]
    df_keep_jerus_merge["speed_to_next_stop"]=df_keep_jerus_merge["speed_to_next_stop"].astype("int32")
    df_keep_jerus_merge["key_id"] = df_keep_jerus_merge['DatedVehicleJourney'].map(str)+df_keep_jerus_merge['StopSequence'].map(str)
    gdf = gpd.GeoDataFrame(df_keep_jerus_merge, geometry=gpd.points_from_xy(df_keep_jerus_merge['Latitude'],df_keep_jerus_merge['Longitude']))
    return gdf

In [ ]:
# def analyseis_compare():
#     df_keep_jerus_merge =compare_to_normal()
#     df_keep_jerus_merge= df_keep_jerus_merge[df_keep_jerus_merge["prop_in_time"]>0.5]
#     df_keep_jerus_merge = df_keep_jerus_merge[df_keep_jerus_merge["tikufim_cat"].isin(["ביקוש בינוני לקו","ביקוש גבוה לקו"])]
# #     df_keep_jerus_merge=df_keep_jerus_merge[(df_keep_jerus_merge["speed_to_next_stop"]-df_keep_jerus_merge["Velocity"])>20 ]
#     return df_keep_jerus_merge
    

In [ ]:
def maps_me(data,name):
    gdf = data
    m = folium.Map(location=[31.775196, 35.206058],tiles='openstreetmap',map_type="roadmap",zoom_start=13,control_scale=True)
    folium.raster_layers.TileLayer('Stamen Toner').add_to(m)
    folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(m)
    fg = folium.FeatureGroup()
    tooltip = "Click me!"
    # get the map content and viz
    for index, x in gdf.iterrows():
        locations = [x['Latitude'], x['Longitude']]
        # html for the popup data
        html_pop = f"""
                    <h1>מס' נסיעה  - {x.DatedVehicleJourney}</h1>
                    <p>קו - {x["קו"]}</p>
                    <p> מס' לוחית רישוי - {x.Vehicle}</p>
                    <p> שם יישוב מוצא - {x["שם יישוב מוצא"]}</p> 
                    <p> מס' אשכול - {x["מס' אשכול"]}</p> 
                    <p> שם תחנה - {x.StopName}</p>
                    <p>מקט קו - {x.Line_code}</p>
                    <p>כיוון - {x.Direction}</p>
                    <p> קוד מפעיל - {x.Operator}</p>  
                    <p>מהירות - {x.Velocity}</p>
                    <p>מרחק לתחנה הבאה - {x.dist_to_next_stop}</p>
                    <p>מרחק מהתחנה הראשונה - {x.DistanceFromStop}</p>
                    <p> סדר תחנות - {x.Order}</p>
                    <p> ביקוש לקו - {x.tikufim_cat}</p>
                    <p> זמן ממוצע מהנורמה לתחנה הבאה - {x.average_time_travel_to_next_stop}</p>
                    <p> סטיית תקן מהנורמה - {x.sd_dif}</p>
                    <p> אחוזי הגעה מהנורמה - {x.prop_in_time}</p>
                    <p> מהירות עד לתחנה הבאה מהנורמה - {x.speed_to_next_stop}</p>
                    <meta http-equiv="refresh" content="20">
                    """
        
        #folium.Marker(location, popup = f'שם תחנה: {x.stop_name}').add_to(m)
        folium.CircleMarker(
            radius=x['Velocity']/7 +8,
            #radius=10,
            location=locations,
            popup = folium.Popup(html_pop, max_width=2650),
            fill=True,
            color=x["color"],
            fill_opacity = 0.7,
            tooltip=tooltip,
            name=name,
            legend=True,).add_to(m)
        
        if(x["color"]== "red"):
            marker = folium.Marker(location=locations,
                               popup=x.Line_code,icon=folium.Icon(icon='bus', prefix='fa'))
            marker.add_to(fg)
        
    
#         else:
#             marker = folium.Marker(location=locations,
#                                popup=x.Line_code)
#             marker.add_to(fg)
            
    fg.add_to(m)
    Search(fg,collapsed=False).add_to(m)
    plugins.Draw(export=True).add_to(m)
        

    list_colors = sorted(list(gdf['color'].unique()))
    # add Title
   
    
    # set the list of colors for the legend
    
#     vis_params = {
#       'palette': list_colors}
#     colors = [color for color in vis_params['palette']]
    
    
#     colormap = LinearColormap(colors=colors)
    loc = name
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 <a href=file:///Z:/SharedFolder/siri_real_time_map/velocitiy_mean_try.html target="_blank"> Click Here: Dashboard Velocitiy mean</a>
                 <meta http-equiv="refresh" content="20">
                 '''.format(loc) 
    # add the layer control widget
    folium.map.LayerControl('topright', collapsed=True).add_to(m)
    m.get_root().html.add_child(folium.Element(title_html))
    m.add_child(MeasureControl())

# If you want get the user device positon after load the map, set auto_start=True
#     plugins.LocateControl(auto_start=True).add_to(m)
    plugins.Geocoder().add_to(m)
#     m.add_child(folium.LatLngPopup())
    
    # folium.Icon(color='blue',icon='bus', prefix='fa').add_child(m)
#     m.add_child(colormap)
    return m


In [ ]:
def caulate_time():
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    return current_time

In [ ]:
def maps_concat_data(): 
    df_all_signals =pd.DataFrame()
    
    while True:
        try:
            current_time = caulate_time()
            df_all_signals = pd.concat([df_all_signals,compare_to_normal()])
            
            df_all_signals =df_all_signals.drop_duplicates(subset =["DatedVehicleJourney","StopSequence"],keep="first")
            df_signal_detect = df_all_signals.groupby(["DatedVehicleJourney"]).size().to_frame("size_trips")["size_trips"] <= 3
            keep_index_refresh = df_signal_detect[df_signal_detect==True].index
            df_all_signals= df_all_signals[df_all_signals["DatedVehicleJourney"].isin(keep_index_refresh)]
            dist = (int(current_time[3:5]) -df_all_signals.groupby(["DatedVehicleJourney"])["RecordedAtTime_onlytime"].last().str[3:5].astype(int))<=3  
            dist_index_refresh =dist[dist==True].index
            df_all_signals= df_all_signals[df_all_signals["DatedVehicleJourney"].isin(dist_index_refresh)]
            df_all_signals["sd_who_you_are"] =0
            df_all_signals["Arrivalminutes"] =pd.to_datetime(df_all_signals.RecordedAtTime_onlytime).dt.minute.astype("int")
            df_all_signals["Arrivalseconds"] = pd.to_datetime(df_all_signals.RecordedAtTime_onlytime).dt.second.astype("int")
            df_all_signals["time_total"] =df_all_signals["Arrivalminutes"]+df_all_signals["Arrivalseconds"]/60 
            df_all_signals =df_all_signals.sort_values(by=["DatedVehicleJourney","StopSequence"])
            df_all_signals['diff_time'] =df_all_signals.groupby("DatedVehicleJourney")['time_total'].diff( periods = 1).fillna(0)
            
    
            #caculate for errors 
        #     df_all_error = pd.concat([df_all_error,df_all_signals])
        #     df_all_error =df_all_error.drop_duplicates(subset =["DatedVehicleJourney","StopSequence"])
        #     df_all_error_detect = df_all_error.groupby(["DatedVehicleJourney"]).size().to_frame("size_trips")["size_trips"] <= 3
        #     keep_index_refresh_error = df_all_error_detect[df_all_error_detect==True].index
        #     df_all_error= df_all_error[df_all_error["DatedVehicleJourney"].isin(keep_index_refresh_error)]
        #     dist_error = (int(current_time[3:5]) -df_all_error.groupby(["DatedVehicleJourney"])["RecordedAtTime_onlytime"].last().str[3:5].astype(int))<=3  
        #     dist_error_index_refresh =dist_error[dist_error==True].index
        #     df_all_error= df_all_error[df_all_error["DatedVehicleJourney"].isin(dist_error_index_refresh)]
            sd_2_ahed = df_all_signals[df_all_signals['diff_time']>(2*(df_all_signals['sd_dif'])+df_all_signals['average_time_travel_to_next_stop']+0.5)]
            sd_3_ahed = df_all_signals[df_all_signals['diff_time']>(3*(df_all_signals['sd_dif'])+df_all_signals['average_time_travel_to_next_stop']+0.5)]
            df_all_signals.loc[sd_3_ahed.index,"sd_who_you_are"] =3
            df_all_signals.loc[sd_2_ahed.index,"sd_who_you_are"] =2
            df_all_signals["color"] ="green"
#             df_all_signals.loc[df_all_signals["מס' אשכול"]==134]["color"] ="pink"
            df_all_signals.loc[df_all_signals[df_all_signals["key_id"].isin(df_all_signals[df_all_signals["sd_who_you_are"]==2]["key_id"])]["key_id"].index, 'color'] ="orange"
            df_all_signals.loc[df_all_signals[df_all_signals["key_id"].isin(df_all_signals[df_all_signals["sd_who_you_are"]==3]["key_id"])]["key_id"].index, 'color'] ="red"
            df_all_signals.loc[df_all_signals[df_all_signals["key_id"].isin(df_all_signals[df_all_signals["diff_time"]==0]["key_id"])]["key_id"].index, 'color'] ="green"
            orange_data = df_all_signals[df_all_signals["color"]=="orange"]
            red_data = df_all_signals[df_all_signals["color"]=="red"]
            t2 = time.localtime()
            current_date = time.strftime("%Y-%m-%d", t2)
            print(f"---time update {current_time}----")
            print(f"amount of all points {df_all_signals.shape[0]}")
            print(f"amount of all orange points {orange_data.shape[0]}")
            print(f"amount of all red points {red_data.shape[0]}")
            print("***************")
            print("**********")
            print("*******")
            print("*****")
            print("***")
            print("**")
            print("*")
            def figures_to_html(figs, filename=r"Z:\SharedFolder\siri_real_time_map\velocitiy_mean_try.html"):
                dashboard = open(filename, 'w')
                dashboard.write("<html><head></head><body>" + "\n")
                for fig in figs:
                    inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
                    dashboard.write(inner_html)
                dashboard.write("""</body><meta http-equiv="refresh" content="36"></html>""" + "\n")

            try:
                plot_data=df_all_signals.loc[df_all_signals["DatedVehicleJourney"].isin(orange_data["DatedVehicleJourney"])]
                plot_data_group_orange = plot_data.groupby(["DatedVehicleJourney","Line_code","קו","Direction","Order"])['Velocity'].mean().to_frame("veloc_mean").reset_index()
                fig =px.bar(plot_data_group_orange,color = plot_data_group_orange.Order,text="Order",labels={'x':'Trip ID',"veloc_mean":"Velocity mean","קו":"Line"},hover_data=["Line_code","קו"],y=plot_data_group_orange.veloc_mean,x=plot_data_group_orange.DatedVehicleJourney.astype(str),title =  f"Velocity mean of line codes that are in the orange group - Time of feed {current_time} and Date {current_date} and orange points {orange_data.shape[0]}")
                fig.update_layout(xaxis_title="TRIP ID",yaxis_title="Velocity mean")
            except:
                    print("sorry no orange data")
            try:
                plot_data_red=df_all_signals.loc[df_all_signals["DatedVehicleJourney"].isin(red_data["DatedVehicleJourney"])]
                plot_data_group_red = plot_data_red.groupby(["DatedVehicleJourney","Line_code","קו","Direction","Order"])['Velocity'].mean().to_frame("veloc_mean").reset_index()
                fig2 =px.bar(plot_data_group_red,color = plot_data_group_red.Order,text="Order",labels={'x':'Trip ID',"veloc_mean":"Velocity mean","קו":"Line"},hover_data=["Line_code","קו"],y=plot_data_group_red.veloc_mean,x=plot_data_group_red.DatedVehicleJourney.astype(str),title = f"Velocity mean of line codes that are in the red group - Time of feed {current_time} and Date {current_date} and red points {red_data.shape[0]}")
                fig2.update_layout(xaxis_title="TRIP ID",yaxis_title="Velocity mean")
            except:
                    print("sorry no red data")
            try:
                plot_data_group_total = df_all_signals.groupby(["מס' אשכול"])['Velocity'].mean().to_frame("veloc_mean").reset_index()
                fig3 =px.bar(plot_data_group_total,y=plot_data_group_total.veloc_mean,labels={'x':'Cluster code',"veloc_mean":"Velocity mean"},x=plot_data_group_total["מס' אשכול"].astype(str),title = "Velocity mean of clusters")
                fig3.update_layout(xaxis_title="CLUSTER ID",yaxis_title="Velocity mean")
            except:
                    print("sorry no data")
            try:
                
                figures_to_html([fig, fig2, fig3])
                
            


                
            except:
                try:
                    figures_to_html([fig, fig3])
                except:
                    try:
                        figures_to_html([fig3])
                    except:
                        print("sorry")
            
            maps_me(df_all_signals,f'REAL TIME SIRI ALL DATA - Time of feed {current_time} and Date {current_date} and total points {df_all_signals.shape[0]} and orange points {orange_data.shape[0]} and red points {red_data.shape[0]}').save(r"G:\python_scripts2\real_time_data_siri_project\GTFS_map.html")
            maps_me(df_all_signals,f'REAL TIME SIRI ALL DATA - Time of feed {current_time} and Date {current_date} total points {df_all_signals.shape[0]} and orange points {orange_data.shape[0]} and red points {red_data.shape[0]}').save(r"Z:\SharedFolder\siri_real_time_map\GTFS_map.html")
            print(f"---> you can refrash the MAP Now New Data has arrive! <----")
            time.sleep(30)
            
        except :
            print("No new data!!!")
            time.sleep(30)


In [ ]:
maps_concat_data()